## Making Keras Model

think about hyper parameters like Filter size, number of filters, which type of padding to use, which activatioon functions to use etc.

## Libraries

In [31]:
#                                                   LIBRARIES
import os

# Image preprocesssing- processing
import cv2
from PIL import Image
import numpy as np

# Neural Network libraries
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout

In [93]:
CSV_INPUT_FOLDER = ".." + os.sep + "csv" + os.sep
TRAIN_CSV = CSV_INPUT_FOLDER + "train_list.csv"
TEST_CSV = CSV_INPUT_FOLDER + "test_list.csv"
VALIDATION_CSV = CSV_INPUT_FOLDER + "validation_list.csv"

In [112]:
def read_csvs(train_csv = TRAIN_CSV, test_csv = TEST_CSV, validation_csv = VALIDATION_CSV):
    train_data = []
    test_data = []
    validation_data = []

    # convert back to numpy array

    train = np.loadtxt(train_csv,dtype=str,
                                     delimiter=',', usecols=(0, 1), unpack=True)

    np.random.shuffle(train)

    train_d, train_labels = train

    for data in train_d:
        data = ".." + data
        img=cv2.imread(data)

        # convert image to array
        img = np.array(img)

        img = Image.fromarray(img, 'RGB')

        train_data.append(np.array(img))

    assert len(train_data) == len(train_labels)


    test_d, test_labels = np.loadtxt(test_csv,dtype=str,
                                     delimiter=',', usecols=(0, 1), unpack=True)
    for data in test_d:
        data = ".." + data
        img=cv2.imread(data)

        # convert image to array
        img = np.array(img)

        img = Image.fromarray(img, 'RGB')

        test_data.append(np.array(img))

    validation_d, validation_labels = np.loadtxt(validation_csv,dtype=str,
                                     delimiter=',', usecols=(0, 1), unpack=True)
    for data in validation_d:
        data = ".." + data
        img=cv2.imread(data)

        # convert image to array
        img = np.array(img)

        img = Image.fromarray(img, 'RGB')

        validation_data.append(np.array(img))

    train_labels = np.array(train_labels)
    test_labels = np.array(test_labels)
    validation_labels = np.array(validation_labels)

    train_data=np.array(train_data)
    test_data=np.array(test_data)
    validation_data=np.array(validation_data)

    return train_data, train_labels, test_data, test_labels, validation_data, validation_labels

train_data, train_labels, test_data, test_labels, validation_data, val_labels = read_csvs(TRAIN_CSV, TEST_CSV, VALIDATION_CSV)

IndexError: tuple index out of range

In [111]:
num_classes = 3
train_data, train_labels, test_data, test_labels, validation_data, val_labels = read_csvs(TRAIN_CSV, TEST_CSV, VALIDATION_CSV)

IndexError: tuple index out of range

In [96]:
def transform_names_into_numbers(train_labels_name, test_labels_name, validation_labels_name):
    train_labels = test_labels = validation_labels = []
    # train_labels
    for idx,label in enumerate(train_labels_name):
        if label == 'Bear':
            train_labels.append(0)
        elif label == 'Chicken':
            train_labels.append(1)
        elif label == 'Turtle':
            train_labels.append(2)

    # test_labels
    for idx,label in enumerate(test_labels_name):
        if label == 'Bear':
            test_labels.append(0)
        elif label == 'Chicken':
            test_labels.append(1)
        elif label == 'Turtle':
            test_labels.append(2)

    # validation_labels
    for idx,label in enumerate(validation_labels_name):
        if label == 'Bear':
            validation_labels.append(0)
        elif label == 'Chicken':
            validation_labels.append(1)
        elif label == 'Turtle':
            validation_labels.append(2)

    train_labels = np.array(train_labels)
    test_labels = np.array(test_labels)
    validation_labels = np.array(validation_labels)

    return train_labels, test_labels, validation_labels

In [97]:
# One hot encoding for labels
train_labels = keras.utils.to_categorical(train_labels,num_classes)
test_labels = keras.utils.to_categorical(test_labels,num_classes)
validation_labels = keras.utils.to_categorical(val_labels,num_classes)

In [98]:
# import sequential model and all the required layers
def create_model():
    #make model
    model=Sequential()
    # Pairs of Conv2D layer and MaxPool2D Layer with increasing filter sizes ( 16,32 ,64). This helps to make image grow more in depthwise and become more flatten.
    # Maxpool: great as they optimize the training time

    # capas de calculo -> ir de menor a mayor, suelen ser potencias / multiplos de 2 (las layers)

    model.add(Conv2D(filters=64,kernel_size=3,padding="same",activation="relu",input_shape=(200,200,3))) # parte imp = relu
    model.add(MaxPooling2D(pool_size=2))

    model.add(Conv2D(filters=128,kernel_size=3,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))

    model.add(Conv2D(filters=256,kernel_size=3,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=4))

    model.add(Dropout(0.2))
    model.add(Flatten()) # without flatten output shape = (50, 50, 6) -> flatten = (None, 6) which we need to get layer output


    # capas de clasificacion

    model.add(Dense(200,activation="relu"))

    # Final dense layer with num_classes nodes = categories of animals we have in the set
    # Softmax activation is used to give scores to these categories which lie between 0 and 1.
    model.add(Dense(num_classes,activation="softmax"))
    model.summary()

    # compile the model
    # We use loss function as categorical_crossentropy and Adam optimizer

    # if binary data -> loss = Binary Cross Entropy and activation = sigmoid
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                      metrics=['accuracy'])

    return model

In [99]:
model = create_model()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 200, 200, 64)      1792      
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 100, 100, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 100, 100, 128)     73856     
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 50, 50, 128)      0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 50, 50, 256)       295168    
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 12, 12, 256)     

In [59]:
# training
# batch potencias de 2, a mayor bacths mayor estabilidad del grandiente
model.fit(train_data,train_labels,batch_size=32 ,epochs=15,verbose=1)

Epoch 1/30
46/46 [==============================] - 68s 1s/step - loss: 41.8497 - accuracy: 0.3973
Epoch 2/30
46/46 [==============================] - 61s 1s/step - loss: 0.9158 - accuracy: 0.5747
Epoch 3/30
46/46 [==============================] - 58s 1s/step - loss: 0.7845 - accuracy: 0.6514
Epoch 4/30
46/46 [==============================] - 57s 1s/step - loss: 0.7097 - accuracy: 0.6932
Epoch 5/30
46/46 [==============================] - 58s 1s/step - loss: 0.5554 - accuracy: 0.7829
Epoch 6/30
46/46 [==============================] - 59s 1s/step - loss: 0.4781 - accuracy: 0.7973
Epoch 7/30
46/46 [==============================] - 59s 1s/step - loss: 0.4114 - accuracy: 0.8212
Epoch 8/30
46/46 [==============================] - 60s 1s/step - loss: 0.4170 - accuracy: 0.8534
Epoch 9/30
46/46 [==============================] - 61s 1s/step - loss: 0.3587 - accuracy: 0.8603
Epoch 10/30
46/46 [==============================] - 62s 1s/step - loss: 0.3101 - accuracy: 0.8760
Epoch 11/30
46/46 

In [60]:
# test the model
score = model.evaluate(test_data, test_labels, verbose=1)
print('\n', 'Test accuracy:', score[1])

6/6 [==============================] - 2s 261ms/step - loss: 2.8495 - accuracy: 0.6319

 Test accuracy: 0.6318681240081787


In [62]:
# save the model

'''# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")'''

model.save('../models/model_200x200.h5') # Saves the entire model to a single artifact


In [100]:
def convert_to_array(img):
    im = cv2.imread(img)
    img = Image.fromarray(im, 'RGB')
    return np.array(img)

def get_animal_name(label):
    if label==0:
        return "Bear"
    if label==1:
        return "Chicken"
    if label==2:
        return "Turtle"

def predict_animal(file):
    print("Predicting .................................")
    # ar = convert_to_array(file)
    file = file/255
    a = []
    a.append(file)
    a = np.array(a)
    score = model.predict(a,verbose=1)
    label_index=np.argmax(score)
    acc=np.max(score)
    animal=get_animal_name(label_index)
    print("The predicted Animal is a "+animal+" with accuracy =    "+str(acc))

    return label_index, animal

In [106]:
def validation(val_data = validation_data, val_lab = val_labels):
    # function not finished and not working
    good_predicted = 0
    bad_predicted = 0
    for idx, data in enumerate(val_data):
        label_index, animal = predict_animal(data)
        if label_index == val_lab[idx]:
            good_predicted += 1
        else:
            bad_predicted += 1

    print("Good predicted: " + str(good_predicted))
    print("Bad predicted: " + str(bad_predicted))

# validation(validation_data, val_labels)

Predicting .................................
1/1 [==============================] - 0s 40ms/step
The predicted Animal is a Bear with accuracy =    0.3789469
Predicting .................................
1/1 [==============================] - 0s 38ms/step
The predicted Animal is a Bear with accuracy =    0.40878782


KeyboardInterrupt: 

In [109]:
# predict image
file = "../datasets/animals/data_resized/Chicken/60.jpg"
file = cv2.imread(file)
predict_animal(file)

Predicting .................................
1/1 [==============================] - 0s 37ms/step
The predicted Animal is a Bear with accuracy =    0.38710973


(0, 'Bear')

In [ ]:
# tensor board -> api to see in real time evolution of the model (from tensorflow)